# Item 17: Prefer `defaultdict` Over `setdefault` to Handle Missing Items in Internal State

In [2]:
# For some usescases, the setdefault method appears to be the shortest option when dealing with missing
# dictionary keys. In this example, we can add new cities to the sets, whether the country name is already
# present in the dictionary or not.
visits = {
    'Mexico': {'Tulum', 'Puerto Vallarta'},
    'Japan': {'Hakone'}
}

visits.setdefault('France', set()).add('Arles') # Short

if (japan := visits.get('Japan')) is None: # Long
    visits['Japan'] = japan = set()
japan.add('Kyoto')
print(visits)

{'Mexico': {'Puerto Vallarta', 'Tulum'}, 'Japan': {'Hakone', 'Kyoto'}, 'France': {'Arles'}}


What about the situation where we *do* control creation of the dictionary being accessed? This is generally the case when we're using a dictionary instanc to keep track of the internal state of a class.

In [5]:
# Here we wrap the code above in a class with helper methods to access the dynamic inner state stored in a 
# dictionary
class Visits:
    def __init__(self) -> None:
        self.data = {}

    def add(self, country, city):
        city_set = self.data.setdefault(country, set())
        city_set.add(city)

In [8]:
# The class above hides the complexity of calling setdefault correctly, and it provides a nicer interface for the
# programmer
visits = Visits()
visits.add('Russia', 'Yekaterinburg')
visits.add('Tanzania', 'Zanibar')
print(visits.data)


{'Russia': {'Yekaterinburg'}, 'Tanzania': {'Zanibar'}}


`Visits.add` is still not ideal because the `setdefault` method is still confusingly named and we have to construct a new `set` instance on every call, thus increasing inefficiency.

Luckily, the `defaultdict` class from the `collectios` built-in module simplifies this common use case by automatically storing a default value when a key doesn't exist. All we have to do is provide a function that will return the default value to use each time a key is missing.

In [9]:
# Here we rewrite the Visits class to use defaultdict
from collections import defaultdict

class Visits:
    def __init__(self) -> None:
        self.data = defaultdict(set)

    def add(self, country, city):
        self.data[country].add(city)

visits = Visits()
visits.add('England', 'Bath')
visits.add('England', 'London')
print(visits.data)

defaultdict(<class 'set'>, {'England': {'London', 'Bath'}})
